In [7]:
import pandas as pd

#baca dataset
data = pd.read_csv('C:\\Users\\ASUS\\Documents\\TUGAS AKHIRKU\\dataujibetul.csv')

# Konversi kolom ke numerik, mengabaikan kesalahan konversi dengan 'errors="coerce"'
data['N'] = pd.to_numeric(data['N'], errors='coerce')
data['P'] = pd.to_numeric(data['P'], errors='coerce')
data['K'] = pd.to_numeric(data['K'], errors='coerce')
data['moisture'] = pd.to_numeric(data['moisture'], errors='coerce')

# Batas tiap n, p, k, dan kelembaban
limits = {
    'N': {'tooLittle': 50, 'idealMin': 50, 'idealMax': 199, 'tooMuch': 200},
    'P': {'tooLittle': 4, 'idealMin': 4, 'idealMax': 13, 'tooMuch': 14},
    'K': {'tooLittle': 50, 'idealMin': 50, 'idealMax': 199, 'tooMuch': 200},
    'moisture': {'tooLittle': 50, 'idealMin': 50, 'idealMax': 80, 'tooMuch': 80}
}

# Fungsi untuk menghitung kebutuhan pupuk
def calculate_fertilizer(n, p, k):
    N_fertilizer = 0
    P_fertilizer = 0
    K_fertilizer = 0
    
    # N
    if n < limits['N']['tooLittle']:
        N_fertilizer = limits['N']['idealMin'] - n
    elif n > limits['N']['tooMuch']:
        N_fertilizer = 0
    elif limits['N']['idealMin'] <= n <= limits['N']['idealMax']:
        N_fertilizer = 0

    # P
    if p < limits['P']['tooLittle']:
        P_fertilizer = limits['P']['idealMin'] - p
    elif p > limits['P']['tooMuch']:
        P_fertilizer = 0
    elif limits['P']['idealMin'] <= p <= limits['P']['idealMax']:
        P_fertilizer = 0

    # K
    if k < limits['K']['tooLittle']:
        K_fertilizer = limits['K']['idealMin'] - k
    elif k > limits['K']['tooMuch']:
        K_fertilizer = 0
    elif limits['K']['idealMin'] <= k <= limits['K']['idealMax']:
        K_fertilizer = 0

    return N_fertilizer, P_fertilizer, K_fertilizer

# Fungsi untuk menghitung kebutuhan air
def calculate_water(moisture):
    water_needed = 0
    
    # Jika kelembaban di bawah batas terlalu rendah, hitung jumlah air yang dibutuhkan
    if moisture < limits['moisture']['tooLittle']:
        water_needed = limits['moisture']['idealMin'] - moisture
    elif moisture > limits['moisture']['tooMuch']:
        water_needed = 0
    elif limits['moisture']['idealMin'] <= moisture <= limits['moisture']['idealMax']:
        water_needed = 0

    # Asumsikan 1% kelembaban = 10 ml air (ini hanya contoh, bisa disesuaikan berdasarkan data sebenarnya)
    water_needed_ml = water_needed * 10
    return water_needed_ml

# Kolom tambahan untuk masing-masing N, P, K, dan kelembaban
data['Urea (N)'] = data.apply(lambda row: calculate_fertilizer(row['N'], row['P'], row['K'])[0], axis=1)
data['SP36 (P)'] = data.apply(lambda row: calculate_fertilizer(row['N'], row['P'], row['K'])[1], axis=1)
data['KCL (K)'] = data.apply(lambda row: calculate_fertilizer(row['N'], row['P'], row['K'])[2], axis=1)
data['Air (ml)'] = data.apply(lambda row: calculate_water(row['moisture']), axis=1)

# Menyimpan dataset baru
data.to_csv('dataset_with_fertilizer_and_water.csv', index=False)

print('Dataset dengan nilai pupuk dan air telah disimpan ke dataset_with_fertilizer_and_water.csv')


Dataset dengan nilai pupuk dan air telah disimpan ke dataset_with_fertilizer_and_water.csv


In [8]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# baca dataset
dataset = pd.read_csv('C:\\Users\\ASUS\\Documents\\TUGAS AKHIRKU\\dataset_with_fertilizer_and_water.csv')

# fitur dan label
X = dataset[['moisture', 'ph', 'N', 'P', 'K']]
y = dataset[['Urea (N)', 'SP36 (P)', 'KCL (K)', 'Air (ml)']]  # Include moisture

# pembagaian dataset untuk training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# membuat model decision tree
model = DecisionTreeRegressor()

# train (latih) model
model.fit(X_train, y_train)

# prediksi data testing
y_pred = model.predict(X_test)

# evaluasi dengan mse
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (MSE) pada data testing: {mse}')

# menyimpan model
joblib.dump(model, 'fertilizer_and_water_model.pkl')

# membaca dataset baru
new_data = pd.read_csv('C:\\Users\\ASUS\\Documents\\TUGAS AKHIRKU\\dataset_with_fertilizer_and_water.csv')

# memprediksi jumlah pupuk dan air yang diperlukan
predicted_values = model.predict(new_data[['moisture', 'ph', 'N', 'P', 'K']])

# mengonversi hasil prediksi ke satuan gram untuk pupuk dan mempertahankan satuan ml untuk air
predicted_fertilizers_grams = predicted_values[:, :3] * 0.5  # Convert only fertilizer columns to grams
predicted_water_ml = predicted_values[:, 3]  # Water column remains in ml

# menambah prediksi ke data baru (array 0,1,2 untuk kolom hasil prediksi pupuk, dan kolom 3 untuk air)
new_data['Urea (N)'] = predicted_fertilizers_grams[:, 0]  # Kolom pertama
new_data['SP36 (P)'] = predicted_fertilizers_grams[:, 1]  # Kolom kedua
new_data['KCL (K)'] = predicted_fertilizers_grams[:, 2]  # Kolom ketiga
new_data['Air (ml)'] = predicted_water_ml  # Kolom keempat

# menampilkan hasil prediksi .3f untuk menampilkan 3 angka di belakang koma
for index, row in new_data.iterrows():
    print(f"Prediksi penambahan pupuk dan air untuk data ke-{index + 1}:")
    print(f"Moisture: {row['moisture']}, pH: {row['ph']}, N: {row['N']}, P: {row['P']}, K: {row['K']}")
    print(f'Urea (N): {row["Urea (N)"]:.2f} gram')
    print(f'SP36 (P): {row["SP36 (P)"]:.2f} gram')
    print(f'KCL (K): {row["KCL (K)"]:.2f} gram')
    print(f'Air: {row["Air (ml)"]:.2f} ml')
    print('---------------------')

# menyimpan data
new_data.to_csv('predicted_fertilizer_and_water.csv', index=False)
print('Prediksi telah disimpan ke predicted_fertilizer_and_water.csv')


Mean Squared Error (MSE) pada data testing: 1.4001746622351163
Prediksi penambahan pupuk dan air untuk data ke-1:
Moisture: 74.0, pH: 6.0, N: 45.0, P: 3.0, K: 53.0
Urea (N): 2.08 gram
SP36 (P): 0.50 gram
KCL (K): 0.00 gram
Air: 0.00 ml
---------------------
Prediksi penambahan pupuk dan air untuk data ke-2:
Moisture: 74.0, pH: 6.0, N: 15.0, P: 0.0, K: 23.0
Urea (N): 20.80 gram
SP36 (P): 0.20 gram
KCL (K): 18.80 gram
Air: 0.00 ml
---------------------
Prediksi penambahan pupuk dan air untuk data ke-3:
Moisture: 74.0, pH: 6.0, N: 19.0, P: 0.0, K: 27.0
Urea (N): 12.90 gram
SP36 (P): 1.19 gram
KCL (K): 9.62 gram
Air: 0.00 ml
---------------------
Prediksi penambahan pupuk dan air untuk data ke-4:
Moisture: 74.0, pH: 6.0, N: 20.0, P: 0.0, K: 28.0
Urea (N): 12.90 gram
SP36 (P): 1.19 gram
KCL (K): 9.62 gram
Air: 0.00 ml
---------------------
Prediksi penambahan pupuk dan air untuk data ke-5:
Moisture: 74.0, pH: 6.0, N: 21.0, P: 0.0, K: 29.0
Urea (N): 12.90 gram
SP36 (P): 1.19 gram
KCL (K): 9.

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

# Misalkan dataset kamu ada dalam DataFrame bernama df
dataset = pd.read_csv('C:\\Users\\ASUS\\Documents\\TUGAS AKHIRKU\\dataset_with_fertilizer_and_water.csv')


X = dataset[['moisture', 'ph', 'N', 'P', 'K']]
y = dataset[['Urea (N)', 'SP36 (P)', 'KCL (K)', 'Air (ml)']] 

# Pisahkan data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Buat dan latih model
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Simpan model
joblib.dump(model, 'C:\\Users\\ASUS\\Documents\\TUGAS AKHIRKU\\fertilizer_and_water_model.pkl')

# Prediksi dengan data uji
y_pred = model.predict(X_test)

# Hitung metrik evaluasi
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R²): {r2}')


Mean Absolute Error (MAE): 0.3416970015552656
Mean Squared Error (MSE): 1.3338525298548887
R-squared (R²): 0.9871614999238554
